In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import json

from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')
pd.set_option('display.max_columns', None)

In [2]:
df=pd.read_csv('../data/interim/spb_house_with_subway.csv')

# feature config

In [3]:
df['date'] = pd.to_datetime(df['date'])

df['YearMonth'] = df['date'].map(lambda x: x.year+(x.month-1)/12)

df['level_proportion'] = (df['level'] - 1) / df['levels']

df.loc[df['rooms']==-1,'rooms'] = 0

df['room_area'] = df['area'] - df['kitchen_area']

studio_index = df.index[df['rooms']==0]
non_studio_index = df.index[df['rooms']!=0]
df.loc[non_studio_index,'area_per_room'] = (df.loc[non_studio_index,'room_area'] 
                                            / df.loc[non_studio_index,'rooms'])
df.loc[studio_index,'area_per_room'] = df.loc[studio_index,'room_area'] 

df.drop(['date', 'time', 'region'], inplace=True, axis=1)

In [4]:
# df.loc[df['SubwayDistance']>=20000, 'StationName'] = 'Без метро'

In [5]:
df.head()

,price,geo_lat,geo_lon,building_type,level,levels,rooms,area,kitchen_area,object_type,StationName,SubwayDistance,YearMonth,level_proportion,room_area,area_per_room
0,6050000,59.805808,30.376141,1,8,10,3,82.6,10.8,1,Купчино,2654.808620,2018.083333,0.700000,71.8,23.933333
1,3600000,59.875526,30.395457,1,2,5,1,31.1,6.0,1,Международная,1078.709945,2018.166667,0.200000,25.1,25.100000
2,3200000,59.827465,30.201827,1,7,9,1,31.0,7.0,1,Проспект Ветеранов,3233.926980,2018.250000,0.666667,24.0,24.000000
3,6500000,59.988334,29.786928,3,3,3,3,89.0,10.0,1,Беговая,23096.539408,2018.250000,0.666667,79.0,26.333333
4,6300000,59.911622,30.284556,3,5,5,2,99.9,14.5,1,Балтийская,1101.143050,2018.250000,0.800000,85.4,42.700000


In [6]:
df = df[(df['price']>1e6) & (df['price']<50e6)].reset_index(drop=True)
df = df[(df['area']>12) & (df['area']<400)].reset_index(drop=True)
df = df[(df['kitchen_area']>1.5) & (df['kitchen_area']<80)].reset_index(drop=True)
df = df[df['rooms']>=0].reset_index(drop=True)
df = df[df['level_proportion']<1].reset_index(drop=True)
df = df[df['room_area']>5].reset_index(drop=True)
df = df[df['area_per_room']>5].reset_index(drop=True)
df.drop(df[(df['price']<6.8e6) & (df['area']>150)].index, inplace=True)
df.describe()

,price,geo_lat,geo_lon,building_type,level,levels,rooms,area,kitchen_area,object_type,SubwayDistance,YearMonth,level_proportion,room_area,area_per_room
count,4.571410e+05,457141.000000,457141.000000,457141.000000,457141.000000,457141.000000,457141.000000,457141.000000,457141.000000,457141.000000,457141.000000,457141.000000,457141.000000,457141.000000,457141.000000
mean,7.681160e+06,59.932740,30.319731,1.948814,7.594716,14.386940,1.818483,57.279613,12.575743,5.288524,2635.071910,2020.006148,0.446958,44.703870,24.602679
std,5.713301e+06,0.084854,0.120430,0.952119,5.703477,7.072323,0.997522,28.136681,6.472380,4.949127,3865.140443,0.761357,0.280683,24.992409,7.198876
min,1.000300e+06,59.647383,29.509349,0.000000,1.000000,1.000000,0.000000,12.100000,1.600000,1.000000,3.013156,2018.083333,0.000000,5.100000,5.057500
25%,4.400000e+06,59.863270,30.256876,1.000000,3.000000,9.000000,1.000000,37.600000,8.500000,1.000000,740.648746,2019.416667,0.200000,26.400000,20.350000
50%,5.910000e+06,59.939084,30.324338,2.000000,6.000000,13.000000,2.000000,50.700000,11.100000,1.000000,1365.721099,2020.083333,0.440000,39.000000,23.800000
75%,8.718270e+06,59.999856,30.399130,3.000000,11.000000,21.000000,2.000000,68.200000,15.010000,11.000000,3006.007484,2020.583333,0.681818,54.600000,27.600000
max,5.000000e+07,60.241984,30.711428,5.000000,35.000000,38.000000,9.000000,390.000000,79.900000,11.000000,46469.507505,2021.333333,0.971429,359.000000,254.000000


In [7]:

# plt.figure(figsize = (20,10))
# sns.regplot(data=df, x='area', y='price')
# plt.axhline(y=6.8e6, color='Green', linestyle='--', linewidth=3)
# plt.axvline(x=150, color='Green', linestyle='--', linewidth=3)

In [8]:
# plt.figure(figsize = (20,5))
# sns.displot(df,x='room_area')

In [9]:
# plt.figure(figsize = (10,5))
# sns.boxplot(data=np.log10(df['room_area']+1),orient='h')

In [10]:
df.head()

,price,geo_lat,geo_lon,building_type,level,levels,rooms,area,kitchen_area,object_type,StationName,SubwayDistance,YearMonth,level_proportion,room_area,area_per_room
0,6050000,59.805808,30.376141,1,8,10,3,82.6,10.8,1,Купчино,2654.808620,2018.083333,0.700000,71.8,23.933333
1,3600000,59.875526,30.395457,1,2,5,1,31.1,6.0,1,Международная,1078.709945,2018.166667,0.200000,25.1,25.100000
2,3200000,59.827465,30.201827,1,7,9,1,31.0,7.0,1,Проспект Ветеранов,3233.926980,2018.250000,0.666667,24.0,24.000000
3,6500000,59.988334,29.786928,3,3,3,3,89.0,10.0,1,Беговая,23096.539408,2018.250000,0.666667,79.0,26.333333
4,6300000,59.911622,30.284556,3,5,5,2,99.9,14.5,1,Балтийская,1101.143050,2018.250000,0.800000,85.4,42.700000


In [11]:
num_categorical_features = ['building_type', 'object_type', 'levels',
                            'level',]
df[num_categorical_features] = df[num_categorical_features].astype(str)
# df = pd.get_dummies(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457141 entries, 0 to 457227
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   price             457141 non-null  int64  
 1   geo_lat           457141 non-null  float64
 2   geo_lon           457141 non-null  float64
 3   building_type     457141 non-null  object 
 4   level             457141 non-null  object 
 5   levels            457141 non-null  object 
 6   rooms             457141 non-null  int64  
 7   area              457141 non-null  float64
 8   kitchen_area      457141 non-null  float64
 9   object_type       457141 non-null  object 
 10  StationName       457141 non-null  object 
 11  SubwayDistance    457141 non-null  float64
 12  YearMonth         457141 non-null  float64
 13  level_proportion  457141 non-null  float64
 14  room_area         457141 non-null  float64
 15  area_per_room     457141 non-null  float64
dtypes: float64(9), int64

In [12]:
X = df.drop('price',axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=2707)



In [13]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer, TransformedTargetRegressor
from sklearn.pipeline import make_pipeline, make_union

categorical_selector = make_column_selector(dtype_include=object)
num_selector = make_column_selector(dtype_exclude=object)
num_features = num_selector(X_train)
categorical_trans = make_column_transformer((OneHotEncoder(),categorical_selector))
categorical_trans


ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f17b7ad5a30>)])

In [14]:
scaler = StandardScaler()
log_transform = FunctionTransformer(np.log1p, validate=True)

num_pipe = make_pipeline(log_transform, scaler)
num_trans = make_column_transformer((num_pipe,num_selector))
num_trans

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('functiontransformer',
                                                  FunctionTransformer(func=<ufunc 'log1p'>,
                                                                      validate=True)),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f17b6055760>)])

In [15]:
preprocessing = make_union(categorical_trans, num_trans)
preprocessing

FeatureUnion(transformer_list=[('columntransformer-1',
                                ColumnTransformer(transformers=[('onehotencoder',
                                                                 OneHotEncoder(),
                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f17b7ad5a30>)])),
                               ('columntransformer-2',
                                ColumnTransformer(transformers=[('pipeline',
                                                                 Pipeline(steps=[('functiontransformer',
                                                                                  FunctionTransformer(func=<ufunc 'log1p'>,
                                                                                                      validate=True)),
                                                                                 ('standardscaler',
                                                                                  StandardScaler())]),
                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f17b6055760>)]))])

In [16]:
# from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, VotingRegressor

model1 = GradientBoostingRegressor(n_estimators=200)

reg_struct = [('reg1', model1),
              ('reg2', 'drop'),
              ('reg3', 'drop')]

regressor = VotingRegressor(estimators=reg_struct, n_jobs=-1, verbose=True)
regressor

/usr/local/lib/python3.8/dist-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/vega/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.3.11py3 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/vega/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/vega/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


VotingRegressor(estimators=[('reg1',
                             GradientBoostingRegressor(n_estimators=200)),
                            ('reg2', 'drop'), ('reg3', 'drop')],
                n_jobs=-1, verbose=True)

In [17]:
pipe = make_pipeline(preprocessing, regressor)
pipe

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('columntransformer-1',
                                                 ColumnTransformer(transformers=[('onehotencoder',
                                                                                  OneHotEncoder(),
                                                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f17b7ad5a30>)])),
                                                ('columntransformer-2',
                                                 ColumnTransformer(transformers=[('pipeline',
                                                                                  Pipeline(steps=[('functiontransformer',
                                                                                                   FunctionTransformer(func=<ufunc 'log1p'>,
                                                                                                                       validate=True)),
                                                                                                  ('standardscaler',
                                                                                                   StandardScaler())]),
                                                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f17b6055760>)]))])),
                ('votingregressor',
                 VotingRegressor(estimators=[('reg1',
                                              GradientBoostingRegressor(n_estimators=200)),
                                             ('reg2', 'drop'),
                                             ('reg3', 'drop')],
                                 n_jobs=-1, verbose=True))])

In [18]:
y_transformer = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

regr = TransformedTargetRegressor(regressor=pipe,
                                  transformer=y_transformer)
regr

TransformedTargetRegressor(regressor=Pipeline(steps=[('featureunion',
                                                      FeatureUnion(transformer_list=[('columntransformer-1',
                                                                                      ColumnTransformer(transformers=[('onehotencoder',
                                                                                                                       OneHotEncoder(),
                                                                                                                       <sklearn.compose._column_transformer.make_column_selector object at 0x7f17b7ad5a30>)])),
                                                                                     ('columntransformer-2',
                                                                                      ColumnTransformer(transformers=[('pipeline',
                                                                                                                       Pipeline(s...
                                                                                                                                       ('standardscaler',
                                                                                                                                        StandardScaler())]),
                                                                                                                       <sklearn.compose._column_transformer.make_column_selector object at 0x7f17b6055760>)]))])),
                                                     ('votingregressor',
                                                      VotingRegressor(estimators=[('reg1',
                                                                                   GradientBoostingRegressor(n_estimators=200)),
                                                                                  ('reg2',
                                                                                   'drop'),
                                                                                  ('reg3',
                                                                                   'drop')],
                                                                      n_jobs=-1,
                                                                      verbose=True))]),
                           transformer=FunctionTransformer(func=<ufunc 'log1p'>,
                                                           inverse_func=<ufunc 'expm1'>))

In [19]:
# regr.fit(X_train,y_train)
# print('Train R2 score: {0:.2f}'.format(regr.score(X_train, y_train)))
# print('Test R2 score: {0:.2f}'.format(regr.score(X_test, y_test)))

In [20]:
from pycaret.regression import *


In [21]:
reg = setup(data = df,            
            target = 'price',
            categorical_features=['building_type', 'object_type', 'levels',
                                  'level', 'StationName', 'rooms'],
#             ordinal_features={'rooms':[0,1,2,3,4,5,6,7,8,9]},
            train_size= 0.75,
            
            preprocess = True,
            
            normalize = True,
            normalize_method='zscore',
            transformation=True,
            
            pca = False,
            pca_components=None,
            
            remove_outliers=False,
            remove_perfect_collinearity=False,
            transform_target=True,
            
            fold_strategy='kfold',
            fold=2,
            
            silent = True,
            )

,Description,Value
0,session_id,5672
1,Target,price
2,Original Data,"(457141, 16)"
3,Missing Values,0
4,Numeric Features,9
5,Categorical Features,6
6,Ordinal Features,0
7,High Cardinality Features,0
8,High Cardinality Method,None
9,Transformed Train Set,"(342855, 174)"


In [22]:
top3 = compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,756960.7478,3013344118083.4297,1735731.0920,0.9076,0.1598,0.0954,65.3100
rf,Random Forest Regressor,820856.2125,3379657693457.0918,1838131.1005,0.8964,0.1665,0.1022,40.7150
catboost,CatBoost Regressor,936142.5666,3757195758080.4546,1938286.9501,0.8848,0.1691,0.1142,3.6100
xgboost,Extreme Gradient Boosting,976885.8674,3908790820509.5474,1976806.7390,0.8802,0.1741,0.1192,87.3650
knn,K Neighbors Regressor,1024566.9174,4551279158484.8955,2133191.6363,0.8605,0.1916,0.1265,175.7100
lightgbm,Light Gradient Boosting Machine,1068942.0485,4658184893818.2627,2158177.1995,0.8572,0.1851,0.1290,1.4150
dt,Decision Tree Regressor,1105251.2092,6372954580231.9805,2524322.4354,0.8047,0.2313,0.1402,1.9550
gbr,Gradient Boosting Regressor,1277005.3647,6658656463472.2842,2580350.9348,0.7959,0.2136,0.1520,20.4650
huber,Huber Regressor,1379052.1299,9449799249300.2910,3073811.8875,0.7102,0.2254,0.1651,13.7400
br,Bayesian Ridge,1400115.9920,10276876233999.2207,3205478.5508,0.6849,0.2257,0.1643,3.2500
